# Expressions régulières en Python

Les explications sont inspirées du site https://docs.python.org/fr/3/howto/regex.html, avec quelques simplifications.

## Principes de base

### Vue d'ensemble

Une expression régulière (souvent appelée _regex_ ou _regexp_) est un concept d'informatique théorique qui possède un grand nombre d'applications pratiques. En très résumé, il s'agit d'un "motif" dans une chaîne de caractères, que l'on peut décrire par une expression formelle. Une recherche de motif qui termine avec un succès s'appellera souvent un _match_, ou une correspondance en bon français.

Il est important de comprendre qu'une même chaîne (ou sous-chaîne) de caractères peut être _matchée_ par plusieurs expressions régulières, certaines simples, d'autres affreusement compliquées. Comme souvent, il faut savoir faire des compromis entre exactitude et simplicité, entre prévoir tous les cas par avance et s'en sortir avec un code lisible par un humain normalement constitué.

Dans toute la suite de ce cours, les regex seront des chaînes préfixées par un `r`. C'est également le préfixe (pour `raw string`) que la librairire `re` utilise pour signaler qu'il s'agit d'une regex et non d'une chaîne standard.

### Premier exemple

Les expressions régulières les plus simples sont celles qui utilisent uniquement des recherches exactes. Prenons le cas d'une liste de mots

```py
["biophore", "voiture", "champion", "boire", "xénobiologiste"]
```
et supposons que l'on recherche les mots contenant `bio`. On a envie dans un premier temps de recourir à la regex `r"bio"`. Cependant, celle-ci ne va ressortir que les _occurrences_ de la sous-chaîne et non les _mots_ contenant la sous-chaîne. Il nous faut un moyen d'exprimer le fait que l'on veuille "n'importe quoi avant, n'importe quoi après, tant que l'on trouve `bio`".

### Joker et répétitions

La question apparue lors du premier exemple se décompose en trois sous-questions. Un "n'importe quoi" (avant ou après `bio`) est une sous-chaîne, formée d'un nombre inconnu de caractères, chacun arbitraire (disons dans une classe assez large) et sans lien avec ses voisins.

- Quel ensemble de caractères est autorisé en dehors de `bio` ?
- Comment la représenter par une expression régulière ?
- Comment exprimer une répétition un nombre arbitraire de fois ?

La première question relève de la modélisation du problème. Suppose-t-on que l'on aura que des mots du dictionnaire ?
- Si oui, c'est une hypothèse forte de structure et on peut drastiquement simplifier nos regex.
- Si non, veut-on prendre le risque d'avoir des matchs parasites ? Quels matchs seront acceptables ?

Nous allons emprunter la voie de la simpliciité en supposant qu'une chaîne comme `9_biows` ne sera pas présente dans notre liste. Nous pouvons ainsi introduire le premier métacaractère de ce cours, le "joker" `.`.

Utilisé dans une regex, un point ne matche pas le caractère `.`, mais n'importe quel caractère (hormis le retour à la ligne, qui est un caractère spécial non imprimable). Ainsi, la chaîne `biophore` est matchée par la régex `r"bio....."`. C'est un progrès, mais ce n'est pas encore très lisible (imaginez compter les points sur votre écran).

Le deuxième métacaractère que l'on introduit est l'astérisque `*`. Il indique que le caractère précédent (ou l'ensemble précédent, quand nous verrons les ensembles) peut apparaître un nombre arbitraire de fois, de $0$ à $+ \infty$. Dans notre exemple, la regex `r"bio.*"` va matcher avec `bio`, `biologique`, `biophore`, mais aussi `biowszno98!$%f`. En revanche, `xénobiologiste` ne sera pas matchée, la regex ne trouvera que la sous-chaîne `biologiste`.

### De la gloutonnnerie en regex

L'usage de la paire de métacaractères `.*` appelle un commentaire. Puisque le nombre de caractères inconnus est arbitraire, faut-il, lorsqu'on passe la chaîne `biologiste` à la regex `r"bio.*"`, renvoyer `bio`, `biologiste`, `biolo` ou toutes les possibilités intermédiaires ? Par défaut, une regex est dite _gloutonne_, c'est-à-dire qu'elle cherche la plus longue sous-chaîne possible pour la regex donnée et ne renvoie que ce résultat. La librairie Python `re`, qui permet de travailler avec des expressions régulières, comporte des options permettant de préciser le comportement attendu (glouton ou économe, seulement le premier résultat ou tous,...). Dans un premier temps, nous laisserons ces possibilités de côté.

### Ensembles

Nous savons à ce stade donner une sous-chaîne exacte et répéter un caractère, éventuellement inconnu, un nombre quelconque de fois. Il serait bon de disposer d'une sorte d'opérateur `OU`, permettant de matcher sur plusieurs possibilités à la fois parmi une liste restreinte, le `.` étant trop peu restrictif. Les ensembles, ou classes, répondent à cette problématique.

Nous introduisons maintenant les métacaractères `[` et `]`. Les caractères qui sont à l'intérieur de ces crochets seront interprétés littéralement, y compris les métacaractères `.` et `*` décjà rencontrés et seront implicitement séparés par un `OU` logique.

Ainsi, la regex `r"[abc]"` permet de matcher les trois chaînes `a`, `b`, `c` et aucune autre. De même, la regex `r"[ab][ab]"` matchera sur les quatre chaînes `aa`, `ab`, `ba`, `bb` et aucune autre.

Comme mentionné brièvement plus haut, le métacaractère `*` permet de répéter le caractère _ou l'ensemble_ qui le précède un nombre arbitraire de fois. Ainsi, la regex `r"[ab]*"` matche sur ` ` (la chaîne vide), `a`, `b`, `aa`, `ab`, `ba`, `bb` et ainsi de suite à l'infini.

Certains ensembles sont d'usage fréquent et ont ainsi acquis des raccourcis agréables.

- Les lettres latines minuscules : l'ensemble `r"[abcdefhijklmnopqrstuvwxyz]"` s'abrège en `r"[a-z]"`.
- De même pour les lettres latines capitales : `r"[ABCDEFGHIJKLMNOPQRSTUVWXYZ]"` s'abrège en `r"[A-Z]"`.
- Les chiffres : `r"[0123456789]"` s'abrège en `r"[0-9]"`.

Le principe d'union s'applique également à ces sous-ensembles. On peut cibler l'ensemble des lettres latines avec `r"[a-zA-Z]"`, ou l'ensemble des caractèrs alphanumériques avec `r"[a-zA-Z0-9]"`. On peut étendre encore la liste si on le désire. Ainsi, la regex `r"[a-z%/;$.]"` cible un sur-ensemble strict de l'alphabet minuscule.

Enfin, on dispose en regex _ensembliste_ d'une négation, représentée par le métacaractère `^` lorsqu'il est le premier caractère de l'ensemble. Quelques exemples.

- `r"[^5]"` matche tout sauf `5`.
- `r"[5^]"` matche `5` ou `^`.
- `r"[^a-zA-Z]"` matche tout caractère non alphabétique.
- `r"[^0-9]"` matche tout caractère non numérique.

## Un peu de pratique

Dans les exercices suivants, on se donne un texte (sous la forme d'une longue chaîne de caractères) duquel on veut extraire certaines sous-chaînes selon un motif précisé dans l'énoncé. Pour chaque exercice, répondre aux deux questions suivantes.

1. Proposer une regex raisonnable modélisant la demande.
2. La tester en s'inspirant du code suivant

   import re \
   regex = r"..." \
   texte = ... \
   sortie = re.findall(regex, texte) \
   sortie

où `texte` est donné dans l'énoncé et `regex` est à trouver par vous-mêmes.

### Exercice 1 : préfixes

On cherche les mots commençant par `bio` dans la chaîne suivante.

    biologie bibliotheque biodégradable robotique

### Réponse : 

On propose tout simplement `r"bio[a-z]*"`. On vérifie que la regex fait bien ce qu'on attend.

In [1]:
import re
texte1 = "biologie bibliotheque biodegradable robotique"
mots = re.findall(r"bio[a-z]*", texte1)
mots

['biologie', 'biodegradable']

Insistons sur un point : trouver "la" bonne regex n'a guère de sens. Une regex est un outil adapté à une situation unique, avec des hypothèses plus ou moins implicites sur la structure de vos données en entrée. Les mots ont ici perdu leurs accents, que se passe-t-il si on les écrit en bon français ?

### Exercice 2 : les nombres

Extraire les nombres du texte suivant.

    Il y a 12 étudiants, 3 professeurs et 2025 ordinateurs.

### Réponse : 

On propose la regex `r"[0-9][0-9]*"`, pour ne pas matcher avec la chaîne vide. On teste immédiatement.

In [2]:
texte2 = "Il y a 12 étudiants, 3 professeurs et 2025 ordinateurs."
nombres = re.findall(r"[0-9][0-9]*", texte2)
nombres

['12', '3', '2025']

Remarque : il existe un métacaractère dédiée au cas où l'on veut "au moins un exemplaire" du caractère/de l'ensemble précédent, il s'agit de `+`. Dans l'exercice 2, la regex `r"[0-9]+"`est strictement équivalente à celle proposée.

### Exercice 3 : les hashtags

Ou mots-croisillons d'après l'Académie Française. Bref. Extraire les hashtags du texte suivant. On rappelle qu'un hashtag valide commence par un `#` et une lettre, mais pas de chiffre.

    J’adore #Python et les #regex, mais pas #123abc.

### Réponse

On propose la régex `r"#[A-Za-z]+"`. On teste immédiatement.

In [3]:
texte3 = "J’adore #Python et les #regex, mais pas #123abc."
hashtags = re.findall(r"#[A-Za-z]+", texte3)
hashtags

['#Python', '#regex']

### Exercice 4 : les dates

Extraire les dates de la chaîne suivante.

    The event is on 2024-12-25 and the deadline is 2025-01-15.

### Réponse

On propose la regex `r"[0-9][0-9][0-9][0-9]-[0-9][0-9]-[0-9][0-9]"`. On vérifie.

In [4]:
texte4 = "The event is on 2024-12-25 and the deadline is 2025-01-15."
dates = re.findall(r"[0-9][0-9][0-9][0-9]-[0-9][0-9]-[0-9][0-9]", texte4)
dates

['2024-12-25', '2025-01-15']

Alors oui, ça marche, mais c'est quand même bien moche. On aurait pu oser une regex comme `r"[0-9]+-[0-9]+-[0-9]+"`, valide ici, mais on va prendre un autre chemin.

Il est temps d'introduire une autre paire de métacaractères, les accolades `{}`. Elles généralisent `*` et `+` en précisant le nombre de répétitions voulues sous la forme d'un entier ou intervalle d'entiers (séparés par une virgule `,`). Dans le cas d'un intervalle d'entiers, une borne inférieure vide est prise égale à $0$, une borne supérieure vide est prise égale à $+\infty$.

On a ainsi `*` == `{}` == `{0,}` et `+` == `{1,}`. Dans notre exemple, la regex longue est strictement équivalente à la regex plus courte `r"[0-9]{4}-[0-9]{2}-[0-9]{2}"`, déjà un peu plus lisible.

### Exercice 5 : les acronymes

Extraire les acronymes de la chaîne suivante. Pour les besoins de l'exercice, un acronyme est un mot d'au moins trois lettres, entièrement en lettres capitales.

    Ce texte contient USA et aussi HTML et PyThOn.

### Réponse

On propose la regex `r"[A-Z]{3,}"`. On teste.

In [5]:
texte5 = "Ce texte contient USA et aussi HTML et PyThOn."
majuscules = re.findall(r"\b[A-Z]{3,}\b", texte5)
majuscules

['USA', 'HTML']

### Exercice 6 : les emails

On souhaite extraire toutes les adresses emails du texte suivant.

    Contactez-nous à contact@example.com ou support@test.org.

### Réponse

Cette fois il faut réfléchir un peu plus longtemps.
- Une adresse email comporte nécessairement une arobase `@`, ce sera notre point de repère.
- Avant l'arobase, le login ou identifiant est un enchaînement quasi arbitraire de caractères, qui peuvent être alphanumériques, mais pas uniquement, certains caractères spéciaux (dont le `+`, pour les alias) sont autorisés (mais pas l'arobase, n'exagérons rien).
- Après l'arobase, le nom de domaine auquel se réfère le login. Ce nom de domaine décrit de façon hiérarchique, de droite à gauche, le serveur à cibler.
  - On commence par le _top-level domain_ ou TLD, qui appartient à une liste restreinte (par une autorité internationale) mais évolutive (on a vu apparaître `.paris` ou `.breizh` aux côtés du `.org` ou du `.fr`). Il n'est pas question d'en dresser une liste exhaustive, on se contentera de préciser qu'ils font au moins deux caractère de long et sont alphabétiques. Notez que le point fait partie intégrante du TLD.
  - On enchaîne avec le domaine (et ses sous-domaines éventuels). Il doit y avoir au moins un niveau (par exemple, `ensae.fr`) mais il peut y en avoir plusieurs (`mail.ensae.fr` est un sous-domaine de `ensae.fr`). Pour simplifier, on considérera qu'il s'agit d'une chaîne arbitraire pouvant contenir des lettres, des points, des chiffres et des tirets.

Avec ces hypothèses, on peut proposer la regex `r"[a-zA-Z0-9+-._]+@[a-zA-Z0-9.-]+[.][a-zA-Z]{2,}"`. Comme vous le voyez, chercher à s'exprimer de façon parfaitement rigoureuse peut demander un effort de réflexion conséquent.

In [6]:
texte6 = "Contactez-nous à contact@example.com ou support@test.org."
emails = re.findall(r"[a-zA-Z0-9+-._]+@[a-zA-Z0-9.-]+[.][a-zA-Z]{2,}", texte6)
emails

['contact@example.com', 'support@test.org']

NB : pour les personnes intéressées, sachez qu'il existe plusieurs regex d'email nettement plus précises, mais aussi plus indigestes à lire. On pourra par exemple se faire une opinion à partir de [ce fil StackOverflow](https://stackoverflow.com/questions/201323/how-can-i-validate-an-email-address-using-a-regular-expression).

## Notions plus avancées

### Retour sur la gloutonnerie

Comme mentionné plus haut, par défaut, une regex va match sur la plus grande sous-chaîne possible. Or, dans certains cas on veut au contraire extraire la sous-chaîne la plus courte possible. Par exemple, en vue d'analyser (_parser_) un langage à balise comme le HTML ou le XML.

On utilise pour cela le métacaractère `?`. Placé après un caractère ou un ensemble, il est équivalent à `{0,1}`, mais ce n'est pas ce qui nous occupera ici. Placé après un comptage (par métacaractère comme `*` ou par accolade comme `{2,5}`), il limite la recherche à la plus petite sous-chaîne possible. 


### Exercice 7 : gloutonnerie ou satiété

Extraire les balises HTML de la chaîne suivante.

    <b>gras</b><i>italique</i>

### Réponse

Comme expliqué plus haut, si on essaye un naïf `r"<.*>"`, le matching glouton va renvoyer la chaîne initiale (elle commence par `<` et finit par `>`, donc est valide et maximale). L'ajout du métacaractère `?` limite sa portée et renvoie ce que l'on voulait.

On propose donc `r"<.*?>` et on teste kes deux options.

In [7]:
text = "<b>gras</b><i>italique</i>"
glouton = re.findall(r"<.*>", text)
restreint = re.findall(r"<.*?>", text)
glouton, restreint

(['<b>gras</b><i>italique</i>'], ['<b>', '</b>', '<i>', '</i>'])

La solution non gloutonne donne le résultat attendu.

### Les groupes

Les groupes sont ce qui s'approche le plus des variables en regex. On crée un groupe à la volée en entourant une sous-partie de la regex avec des parenthèses. Créer un groupe permet
- de séparer les groupes dans la sortie de `re.findall()`;
- d'appliquer un opérateur de comptage (comme `*`, `?` ou `+`) à toute la sous-chaîne capturée par le groupe;
- d'appeler les groupes comme des variables à l'intérieur de la même regex.

Les groupes reçoivent implicitement un numéro dans leur ordre d'apparition dans la regex, à partir de $1$. On fait référence au premier groupe parenthésé par `\1`, au deuxième groupe parenthésé par `\2` et ainsi de suite.

Illustrons pas à pas ces notions, dans l'ordre. Reprenons notre exemple de capture de dates vu précédemment.

### Exercice 8 : le retour des dates

Extraire les dates dans la chaîne ci-dessous, d'abord en un bloc, ensuite avec de la capture de groupes.

    Mon anniversaire est le 06/10/2025, et le tien le 25/12/1999.

### Réponse

Les dates ont un format à peine différent de la première fois. On propose la regex `r"[0-9]{2}/[0-9]{2}/[0-9]{4}"` pour la capture d'ensemble, puis `r"([0-9]{2})/([0-9]{2})/([0-9]{4})` pour la capture par groupes et on teste.

In [8]:
texte8 = "Mon anniversaire est le 06/10/2025, et le tien le 25/12/1999."
dates_ensemble = re.findall(r"[0-9]{2}/[0-9]{2}/[0-9]{4}", texte8)
dates_groupes = re.findall(r"([0-9]{2})/([0-9]{2})/([0-9]{4})", texte8)
dates_ensemble, dates_groupes

(['06/10/2025', '25/12/1999'], [('06', '10', '2025'), ('25', '12', '1999')])

On peut ainsi trier et filtrer les groupes capturés, par exemple pour ne garder que les couples (mois, année), renverser l'ordre des éléments, changer le délimiteur et bien d'autres choses.

Dans un autre esprit, supposons vouloir détecter les palindromes dans une liste de mots de $5$ lettres. En Python, on peut évidemment renverser la chaîne de caractères et tester l'égalité, mais restons dans le paradigme des regex pour le moment.

### Exercice 9 : palindromes à $5$ lettres

Extraire les palindromes de la chaîne suivante.

    kayak radar level python regex

### Réponse

Un palindrome à $5$ lettres doit suivre le motif `abcba`, avec `a, b, c` des lettres quelconques de l'alphabet. Il nous faut exprimer les contraintes d'égalité en termes de regex, ce qui se fait en capturant les premières lettres en tant que groupes avant de les appeler au bon endroit.

On propose donc la regex `r"([a-z])([a-z])([a-z])\2\1"` et on la teste.

In [9]:
texte9 = "kayak radar level python regex"
palindromes = re.findall(r"([a-z])([a-z])([a-z])\2\1", texte9)
# On reconstitue les mots trouvés
res = ["".join(match) + match[1] + match[0] for match in palindromes]
res

['kayak', 'radar', 'level']

Jusqu'ici, on a soigneusement évité les caractères non imprimables et blancs, leur invisibilité patente rendant la compréhension plus difficile au début. Il est temps de s'y frotter, en commençant par les caractères d'arrêt `^`, `$`, `\b` et `\n`.

- `^` et `$` indiquent respectivement le début et la fin d'une ligne dans un fichier. Dans nos exemple, ils ne seront pas utilisés.
- `\b` est un séparateur, un caractère de longueur $0$ qui matche avec l'intervalle entre un caractère dans l'ensemble "alphanumérique ou `_`" et un caractère dans l'ensemble complémentaire.
- `\n` est le caractère non imprimable de retour à la ligne.

### Exercice 10 : détection de répétitions

Extraire les mots répétés dans la chaîne suivante.

    This is is a test that that is correct.

### Réponse

Les mots ici sont presque tous en minuscules, on peut estimer que l'on saura trouver à l'oeil une répétition en début de phrase (ou après n'importe quel mot capitalisé). On va donc juger qu'un mot est défini par la regex `r"[a-z]+"`.

Comme on veut tester une répétition, on aura besoin de capturer ce premier mot et de l'appeler avec `\1`. Une regex naïve serait alors `r"([a-z]+)[ ]\1"`. Cette regex détecte correctement la répétition du `is`, mais ne voit que la répétition du `t` entre les deux `that`, au lieu des `that` eux-mêmes (testez !).

Ce comportement semble en contradiction avec l'exercice 7 sur les balises HTML. La nuance avec cet exercice est qu'ici nous avons ajouté une contrainte, matérialisée par l'appel à `\1`. Dans ce cas, l'algorithme de matching va au contraire chercher une solution minimale.

Pour cette raison, on a besoin de contraindre la détection en ajoutant le délimiteur `\b`, pour être certain de couper sur un "vrai" mot. On amende donc la regex en `r"\b([a-z]+)[ ]\1\b"` et on teste.

In [10]:
texte10 = "This is is a test that that is correct."
repetitions = re.findall(r"\b([a-z]+)[ ]\1\b", texte10)
repetitions

['is', 'that']

Remarque : on a volontairement choisi un exemple simple où les mots répétés sont tous en minuscules. Si l'on considère la chaîne modifiée,

    This is is a test. That that is correct.

il faut indiquer à `re` que l'on souhaite ignorer la capitalisation. On peut par exemple passer la chaîne en minuscules avec `.lower()` avant de la donner à `re.findall()`. Il existe des flags (arguments optionnels) que l'on peut passer à `re.findall()`, dont `re.IGNORECASE`, mais les flags ne seront pas abordés ici.

Abordons maintenant les abréviations d'ensembles courants. On a déjà vu un premier niveau d'abréviation, avec `[a-z]` pour les lettres latines minuscules ou `[0-9]` pour les chiffres. Il existe des abréviations encore plus courtes pour ces ensembles, rendant les regex complexes plus compactes. Celles-ci viennent par paire, le pendant capitalisé étant la négation (le complémentaire) de son pendant non capitalisé.

- `\d` : caractère numérique, équivalent à [0-9].
- `\D` : caractère non numérique, équivalent à [^0-9].
- `\w` : caractère alphanumérique, équivalent à [a-zA-Z0-9_].
- `\W` : caractère non-alphanumérique, équivalent à [^a-zA-Z0-9_].
- `\s` : caractère « blanc », équivalent à la classe [ \t\n\r\f\v].
- `\S` : caractère non « blanc », équivalent à la classe [^ \t\n\r\f\v].

On a déjà mentionné `\n` plus haut. Sachez simplement que `\t` est la tabulation. Les autres caractères sont tellement peu utilisés qu'on ne les mentionnera pas.

### Exercice 11 : paires nom:valeur

Extraire séparément noms et valeurs dans la chaîne suivante.

    user:alice id:42 email:alice@example.com

### Réponse

Dans une structure de données de type dictionnaire (incluant les JSON, YAML et autres formats voisins), une clé ne peut pas être de n'importe quel type. En général, il s'agit d'une chaîne ou d'un entier et quoi qu'il en soit, de caractères alphanumériques. Les valeurs, elles, sont un peu plus libres. On fera l'hypothèse (relâchée) qu'elles sont constituées de caractères non blancs.

En se souvenant que l'on veut capturer séparément clés et valeurs, on propose donc la regex `r"(\w+):(\S+)"` et on teste.

In [11]:
texte11 = "user:alice id:42 email:alice@example.com"
paires = re.findall(r"(\w+):(\S+)", texte11)
paires

[('user', 'alice'), ('id', '42'), ('email', 'alice@example.com')]

### Substitution de chaînes

On a pour l'instant uniquement fait de la recherche de regex, mais il peut être intéressant de pouvoir travailler directement avec ce que l'on a détecté, par exemple en remplaçant la sous-chaîne trouvée par une autre, ou en reformatant les données. On utilise pour cela la fonction `re.sub()`, qui s'utilise avec la syntaxe `re.sub(regex à chercher, substitution, chaîne cible)`. Par comparaison, la fonction `re.findall()` possède la syntaxe `re.findall(regex à chercher, chaîne cible)`.

### Exercice 12 : nettoyer un code HTML, version simple

Supprimer toutes les balises pour ne garder que le texte.

    <p>Ceci est <b>important</b></p>

### Réponse

Supprimer les balises HTML revient à les substituer par la chaîne vide `r""`. On a déjà vu plus haut comment détecter de façon non gloutonne les balises HTML, on peut donc réutiliser le code.

In [12]:
texte12 = "<p>Ceci est <b>important</b></p>"
html_nettoye = re.sub(r"<.*?>", r"", texte12)
html_nettoye

'Ceci est important'

### Exercice 13 : nettoyer une adresse IPv4

Une adresse IPv4 est formée de quatre nombres entre $0$ et $255$ séparés par un point. Il est possible, mais inutile, d'écrire chaque nombre avec trois chiffres, en notant les $0$ initiaux.

Simplifier l'adresse IPv4 suivante en retirant les $0$ en début de triplet.

    216.08.094.196

### Réponse

Les frontières entre les nombres et les points sont détectées par le délimiteur `\b`, il va être pratique de s'en servir. Grâce à lui, on peut traiter chaque nombre composant l'IP indépendamment, ce qui est bien plus simple. Pour chaque nombre, on sépare les chiffres significatifs des zéros, on les capture dans un groupe, puis on les substitue à l'ancien nombre.

On propose donc `r"\b0*(\d+)\b"` comme regex de détection et `r"\1"` comme regex de substitution, puis on teste.

In [13]:
ip = "216.08.094.196"
string = re.sub(r'\b0*(\d+)\b', r'\1', ip)
print(string)

216.8.94.196


Remarque : les personnes plus motivées peuvent tenter de nettoyer de la même manière une IPv6 en allant lire [une documentation](https://www.ibm.com/docs/en/i/7.5.0?topic=concepts-ipv6-address-formats). C'est nettement moins simple, mais ça se fait. On peut se satisfaire d'un raccoucissement intermédiaire, en laissant des `:0:` partout sans chercher à atteindre l'état final avec un `::`.

### Exercice 14 : noms de variables autorisés en Python
Un identifiant Python : commence par une lettre ou `_,` suivi de lettres, chiffres ou `_`. Déterminer avec une regex si les identifiants suivants sont valides.

    variable1 _private Data2 2fast var-name if

### Réponse

Naïvement, on a envie de proposer la regex `r"[a-zA-Z_]\w*"`. Or, une fois testée, on s'aperçoit qu'elle matche `fast` en éliminant le `2` et qu'elle coupe `var-name` en `var` et `name`.

On peut régler le cas de `2fast` en ajoutant un délimiteur de mot `\b`, mais il reste le problème de `var-name`. Pour terminer cet exercice il faudrait introduire les autres fonctions du `?`, trop avancées pour ce cours. Une solution est laissée pour les plus curieux d'entre vous.

In [14]:
texte14 = "variable1 _private Data2 2fast var-name if"
valides = re.findall(r"(?:^|\s)([a-zA-Z_]\w*)(?=\s|$)", texte14)
valides

['variable1', '_private', 'Data2', 'if']